# Getting started

### CLEF 2025 - CheckThat! Lab  - Task 4 Scientific Web Discourse - Subtask 4b (Scientific Claim Source Retrieval)

This notebook enables participants of subtask 4b to quickly get started. It includes the following:
- Code to upload data, including:
    - code to upload the collection set (CORD-19 academic papers' metadata)
    - code to upload the query set (tweets with implicit references to CORD-19 papers)
- Code to run a baseline retrieval model (BM25)
- Code to evaluate the baseline model

Participants are free to use this notebook and add their own models for the competition.

# 1) Importing data

In [6]:
import numpy as np
import pandas as pd

## 1.a) Import the collection set
The collection set contains metadata of CORD-19 academic papers.

The preprocessed and filtered CORD-19 dataset is available on the Gitlab repository here: https://gitlab.com/checkthat_lab/clef2025-checkthat-lab/-/tree/main/task4/subtask_4b

Participants should first download the file then upload it on the Google Colab session with the following steps.


In [7]:
# 1) Download the collection set from the Gitlab repository: https://gitlab.com/checkthat_lab/clef2025-checkthat-lab/-/tree/main/task4/subtask_4b
# 2) Drag and drop the downloaded file to the "Files" section (left vertical menu on Colab)
# 3) Modify the path to your local file path
PATH_COLLECTION_DATA = '../subtask4b_collection_data.pkl' #MODIFY PATH

In [8]:
df_collection = pd.read_pickle(PATH_COLLECTION_DATA)

In [9]:
df_collection.info()

<class 'pandas.core.frame.DataFrame'>
Index: 7718 entries, 162 to 1056448
Data columns (total 17 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   cord_uid          7718 non-null   object        
 1   source_x          7718 non-null   object        
 2   title             7718 non-null   object        
 3   doi               7677 non-null   object        
 4   pmcid             4959 non-null   object        
 5   pubmed_id         6233 non-null   object        
 6   license           7718 non-null   object        
 7   abstract          7718 non-null   object        
 8   publish_time      7715 non-null   object        
 9   authors           7674 non-null   object        
 10  journal           6668 non-null   object        
 11  mag_id            0 non-null      float64       
 12  who_covidence_id  528 non-null    object        
 13  arxiv_id          20 non-null     object        
 14  label             7718 n

In [10]:
df_collection.head()

,cord_uid,source_x,title,doi,pmcid,pubmed_id,license,abstract,publish_time,authors,journal,mag_id,who_covidence_id,arxiv_id,label,time,timet
162,umvrwgaw,PMC,Professional and Home-Made Face Masks Reduce E...,10.1371/journal.pone.0002618,PMC2440799,18612429,cc-by,BACKGROUND: Governments are preparing for a po...,2008-07-09,"van der Sande, Marianne; Teunis, Peter; Sabel,...",PLoS One,NaN,NaN,NaN,umvrwgaw,2008-07-09,1215561600
611,spiud6ok,PMC,The Failure of R (0),10.1155/2011/527610,PMC3157160,21860658,cc-by,"The basic reproductive ratio, R (0), is one of...",2011-08-16,"Li, Jing; Blakeley, Daniel; Smith?, Robert J.",Comput Math Methods Med,NaN,NaN,NaN,spiud6ok,2011-08-16,1313452800
918,aclzp3iy,PMC,Pulmonary sequelae in a patient recovered from...,10.4103/0970-2113.99118,PMC3424870,22919170,cc-by-nc-sa,The pandemic of swine flu (H1N1) influenza spr...,2012,"Singh, Virendra; Sharma, Bharat Bhushan; Patel...",Lung India,NaN,NaN,NaN,aclzp3iy,2012-01-01,1325376000
993,ycxyn2a2,PMC,What was the primary mode of smallpox transmis...,10.3389/fcimb.2012.00150,PMC3509329,23226686,cc-by,The mode of infection transmission has profoun...,2012-11-29,"Milton, Donald K.",Front Cell Infect Microbiol,NaN,NaN,NaN,ycxyn2a2,2012-11-29,1354147200
1053,zxe95qy9,PMC,"Lessons from the History of Quarantine, from P...",10.3201/eid1902.120312,PMC3559034,23343512,no-cc,"In the new millennium, the centuries-old strat...",2013-02-03,"Tognotti, Eugenia",Emerg Infect Dis,NaN,NaN,NaN,zxe95qy9,2013-02-03,1359849600


## 1.b) Import the query set

The query set contains tweets with implicit references to academic papers from the collection set.

The preprocessed query set is available on the Gitlab repository here: https://gitlab.com/checkthat_lab/clef2025-checkthat-lab/-/tree/main/task4/subtask_4b

Participants should first download the file then upload it on the Google Colab session with the following steps.

In [11]:
# 1) Download the query tweets from the Gitlab repository: https://gitlab.com/checkthat_lab/clef2025-checkthat-lab/-/tree/main/task4/subtask_4b?ref_type=heads
# 2) Drag and drop the downloaded file to the "Files" section (left vertical menu on Colab)
# 3) Modify the path to your local file path
PATH_QUERY_TRAIN_DATA = '../subtask4b_query_tweets_train.tsv' #MODIFY PATH
PATH_QUERY_DEV_DATA = '../subtask4b_query_tweets_dev.tsv' #MODIFY PATH

In [12]:
df_query_train = pd.read_csv(PATH_QUERY_TRAIN_DATA, sep = '\t')
df_query_dev = pd.read_csv(PATH_QUERY_DEV_DATA, sep = '\t')

In [13]:
df_query_train.head()

,post_id,tweet_text,cord_uid
0,0,Oral care in rehabilitation medicine: oral vul...,htlvpvz5
1,1,this study isn't receiving sufficient attentio...,4kfl29ul
2,2,"thanks, xi jinping. a reminder that this study...",jtwb17u8
3,3,Taiwan - a population of 23 million has had ju...,0w9k8iy1
4,4,Obtaining a diagnosis of autism in lower incom...,tiqksd69


In [14]:
df_query_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12853 entries, 0 to 12852
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   post_id     12853 non-null  int64 
 1   tweet_text  12853 non-null  object
 2   cord_uid    12853 non-null  object
dtypes: int64(1), object(2)
memory usage: 301.4+ KB


In [15]:
df_query_dev.head()

,post_id,tweet_text,cord_uid
0,16,covid recovery: this study from the usa reveal...,3qvh482o
1,69,"""Among 139 clients exposed to two symptomatic ...",r58aohnu
2,73,I recall early on reading that researchers who...,sts48u9i
3,93,You know you're credible when NIH website has ...,3sr2exq9
4,96,Resistance to antifungal medications is a grow...,ybwwmyqy


In [16]:
df_query_dev.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1400 entries, 0 to 1399
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   post_id     1400 non-null   int64 
 1   tweet_text  1400 non-null   object
 2   cord_uid    1400 non-null   object
dtypes: int64(1), object(2)
memory usage: 32.9+ KB


# 2) Running the baseline
The following code runs a BM25 baseline.


In [45]:
from rank_bm25 import BM25Okapi

In [46]:
# Create the BM25 corpus
corpus = df_collection[:][['title', 'abstract']].apply(lambda x: f"{x['title']} {x['abstract']}", axis=1).tolist()
cord_uids = df_collection[:]['cord_uid'].tolist()
tokenized_corpus = [doc.split(' ') for doc in corpus]
bm25 = BM25Okapi(tokenized_corpus)

In [47]:
def get_top_cord_uids(query):
  text2bm25top = {}
  if query in text2bm25top.keys():
      return text2bm25top[query]
  else:
      tokenized_query = query.split(' ')
      doc_scores = bm25.get_scores(tokenized_query)
      indices = np.argsort(-doc_scores)[:20]
      bm25_topk = [cord_uids[x] for x in indices]

      text2bm25top[query] = bm25_topk
      return bm25_topk


In [ ]:
# Retrieve topk candidates using the BM25 model
tqdm.pandas()
df_query_train['bm25_topk'] = df_query_train['tweet_text'].progress_apply(lambda x: get_top_cord_uids(x))
df_query_dev['bm25_topk'] = df_query_dev['tweet_text'].progress_apply(lambda x: get_top_cord_uids(x))

100%|██████████| 1400/1400 [01:47<00:00, 13.04it/s]


In [21]:
df_query_train['bm25_topk'][1]

['2cwvga0k', '33znyrn8', 'st3fyb64', '74dw6emg', 'b68c8qwm']

# 3) Evaluating the baseline
The following code evaluates the BM25 retrieval baseline on the query set using the Mean Reciprocal Rank score (MRR@5).

In [50]:
# Evaluate retrieved candidates using MRR@k
def get_performance_mrr(data, col_gold, col_pred, list_k = [1, 5, 10]):
    d_performance = {}
    for k in list_k:
        data["in_topx"] = data.apply(lambda x: (1/([i for i in x[col_pred][:k]].index(x[col_gold]) + 1) if x[col_gold] in [i for i in x[col_pred][:k]] else 0), axis=1)
        #performances.append(data["in_topx"].mean())
        d_performance[k] = data["in_topx"].mean()
    return d_performance


In [51]:
results_train = get_performance_mrr(df_query_train, 'cord_uid', 'bm25_topk')
results_dev = get_performance_mrr(df_query_dev, 'cord_uid', 'bm25_topk')
# Printed MRR@k results in the following format: {k: MRR@k}
print(f"Results on the train set: {results_train}")
print(f"Results on the dev set: {results_dev}")

Results on the train set: {1: np.float64(0.5081303975725512), 5: np.float64(0.5509777224513084), 10: np.float64(0.5509777224513084)}
Results on the dev set: {1: np.float64(0.505), 5: np.float64(0.5519166666666666), 10: np.float64(0.5573010204081632)}


# 4) Exporting results to prepare the submission on Codalab

In [24]:
df_query_dev['preds'] = df_query_dev['bm25_topk'].apply(lambda x: x[:5])

In [25]:
df_query_dev[['post_id', 'preds']].to_csv('predictions.tsv', index=None, sep='\t')

In [26]:
df_query_dev.head()

,post_id,tweet_text,cord_uid,bm25_topk,in_topx,preds
0,16,covid recovery: this study from the usa reveal...,3qvh482o,"[25aj8rj5, gatxuwz7, 59up4v56, styavbvi, 6sy80...",0.0,"[25aj8rj5, gatxuwz7, 59up4v56, styavbvi, 6sy80..."
1,69,"""Among 139 clients exposed to two symptomatic ...",r58aohnu,"[r58aohnu, p0kg6dyz, 9dlaaye8, iu1d9i57, d06np...",1.0,"[r58aohnu, p0kg6dyz, 9dlaaye8, iu1d9i57, d06np..."
2,73,I recall early on reading that researchers who...,sts48u9i,"[tz2shoso, o877uul1, m1sf159a, sgo76prc, gruir...",0.0,"[tz2shoso, o877uul1, m1sf159a, sgo76prc, gruir..."
3,93,You know you're credible when NIH website has ...,3sr2exq9,"[3sr2exq9, hgpiig0g, sv48gjkk, k0f4cwig, ihgxt...",1.0,"[3sr2exq9, hgpiig0g, sv48gjkk, k0f4cwig, ihgxt..."
4,96,Resistance to antifungal medications is a grow...,ybwwmyqy,"[lzddnb8j, ouvq2wpq, sxx3yid9, vabb2f26, y9fqa...",0.0,"[lzddnb8j, ouvq2wpq, sxx3yid9, vabb2f26, y9fqa..."


In [27]:
df_query_train.head()

,post_id,tweet_text,cord_uid,bm25_topk,in_topx
0,0,Oral care in rehabilitation medicine: oral vul...,htlvpvz5,"[htlvpvz5, h7hj64q5, rwgqkow3, dbgtslc8, am11y...",1.000000
1,1,this study isn't receiving sufficient attentio...,4kfl29ul,"[2cwvga0k, 33znyrn8, st3fyb64, 74dw6emg, b68c8...",0.000000
2,2,"thanks, xi jinping. a reminder that this study...",jtwb17u8,"[8hkxbxz9, w98847ai, jtwb17u8, iy1enazk, pg0l9...",0.333333
3,3,Taiwan - a population of 23 million has had ju...,0w9k8iy1,"[32ua8wb6, zxe95qy9, 30pl5tx3, l4y7v729, b97ac...",0.000000
4,4,Obtaining a diagnosis of autism in lower incom...,tiqksd69,"[tiqksd69, k7smwz6w, b0dzhsrh, yc7cvbii, iiafu...",1.000000


## 5) Re-ranking approach

In [60]:
from sentence_transformers import CrossEncoder
from tqdm import tqdm

# Initialize Cross-Encoder once
cross_encoder = CrossEncoder('cross-encoder/ms-marco-TinyBERT-L2-v2', max_length=512)

def rank_chunks_with_cross_encoder(query, retrieved_chunks, top_k=5):
    """
    Rank the retrieved chunks based on their relevance to the query using a Cross-Encoder.
    """
    query_chunk_pairs = [[query, chunk] for chunk in retrieved_chunks]
    scores = cross_encoder.predict(query_chunk_pairs)
    ranked_chunks = sorted(zip(retrieved_chunks, scores), key=lambda x: x[1], reverse=True)
    return ranked_chunks[:top_k]

# Initialize new column
df_query_dev["cross_encoder_topk"] = None

# Loop over the rows to fill in reranked results
for idx, row in tqdm(df_query_dev.iterrows(), total=len(df_query_dev)):
    query_text = row["tweet_text"]
    topk_ids = row["bm25_topk"]

    chunk_to_uid = {}
    for uid in topk_ids:
        paper_row = df_collection.loc[df_collection["cord_uid"] == uid]
        if paper_row.empty:
            continue

        title = paper_row["title"].values[0] or ""
        abstract = paper_row["abstract"].values[0] or ""
        full_text = f"{title.strip()}\n{abstract.strip()}"
        chunk_to_uid[full_text] = uid

    reranked_chunks = rank_chunks_with_cross_encoder(query_text, list(chunk_to_uid.keys()))
    reranked_uids = [chunk_to_uid[chunk] for chunk, _ in reranked_chunks]

    # Assign directly to the dataframe
    df_query_dev.at[idx, "cross_encoder_topk"] = reranked_uids

# Preview result
print(df_query_dev[["cord_uid", "cross_encoder_topk"]].head())



100%|██████████| 1400/1400 [04:02<00:00,  5.78it/s]

   cord_uid                                 cross_encoder_topk
0  3qvh482o  [o4vvlmr4, hg3xpej0, 25aj8rj5, 59up4v56, bqn29...
1  r58aohnu  [r58aohnu, tu1vevx9, yrowv62k, mcmqx7og, zd8c1...
2  sts48u9i  [osct0lj6, o47v5vgw, sts48u9i, gruir7aw, o877u...
3  3sr2exq9  [kdegnr6i, 3sr2exq9, k0f4cwig, sv48gjkk, z795y...
4  ybwwmyqy  [ybwwmyqy, vabb2f26, ouvq2wpq, sxx3yid9, lzddn...


In [61]:
results_reranked = get_performance_mrr(df_query_dev, 'cord_uid', 'cross_encoder_topk')
print(f"Results on the reranked set: {results_reranked}")

Results on the reranked set: {1: np.float64(0.55), 5: np.float64(0.5952857142857143), 10: np.float64(0.5952857142857143)}


In [59]:
from sklearn.preprocessing import MinMaxScaler
import numpy as np
from sentence_transformers import SentenceTransformer, util


corpus = df_collection.progress_apply(lambda x: f"{x['title']} {x['abstract']}", axis=1).tolist()
cord_uids = df_collection['cord_uid'].tolist()

# Encode corpus once
bi_encoder = SentenceTransformer("all-MiniLM-L6-v2")
corpus_embeddings = bi_encoder.encode(corpus, convert_to_tensor=True)

def hybrid_retrieve(query, bm25, corpus_embeddings, cord_uids, top_k=20):
    # Step 1: Get BM25 scores
    bm25_scores = bm25.get_scores(query.split())  # shape: (N,)

    # Step 2: Get dense (bi-encoder) similarity scores
    query_embedding = bi_encoder.encode(query, convert_to_tensor=True)
    dense_scores = util.cos_sim(query_embedding, corpus_embeddings)[0].cpu().numpy()  # shape: (N,)

    # Step 3: Normalize both score types
    scaler = MinMaxScaler()
    combined_scores = scaler.fit_transform(np.vstack([bm25_scores, dense_scores]).T)  # shape: (N, 2)
    hybrid_scores = combined_scores.sum(axis=1)  # shape: (N,)

    # Step 4: Get top-k documents based on hybrid score
    top_indices = np.argsort(-hybrid_scores)[:top_k]
    top_docs = [cord_uids[i] for i in top_indices]

    return top_docs



100%|██████████| 7718/7718 [00:00<00:00, 125024.19it/s]


In [62]:
df_query_dev["hybrid_topk"] = None

# Apply hybrid retrieval to each row
for idx, row in tqdm(df_query_dev.iterrows(), total=len(df_query_dev)):
    query_text = row["tweet_text"]
    top_docs = hybrid_retrieve(query_text, bm25, corpus_embeddings, cord_uids, top_k=20)
    df_query_dev.at[idx, "hybrid_topk"] = top_docs

100%|██████████| 1400/1400 [02:18<00:00, 10.13it/s]


In [63]:
df_query_dev["hybrid_topk"]

0       [25aj8rj5, styavbvi, jrqlhjsm, hg3xpej0, o4vvl...
1       [r58aohnu, yrowv62k, d06npvro, mcmqx7og, mz1bo...
2       [sts48u9i, 8m59whf9, gruir7aw, qkg8fwbp, osct0...
3       [3sr2exq9, k0f4cwig, z795y51f, sv48gjkk, 4fzzo...
4       [ybwwmyqy, lzddnb8j, vabb2f26, ouvq2wpq, sxx3y...
                              ...                        
1395    [0gn3b98n, 0e97tdez, soa71kwx, d8x3b9a3, n5sei...
1396    [92z77upt, s1qnjgev, 763v4duh, 64ugdvfe, q10pv...
1397    [qn6wawxk, 059oj76m, rtus1qyw, wrsk5vh9, t8jli...
1398    [47p4g80o, 2tax8ajw, rfv0omd6, unl3n0tn, wt6az...
1399    [nih4l4ok, w4dyadpa, dfhxcqr8, p82l6d9a, msut5...
Name: hybrid_topk, Length: 1400, dtype: object

In [ ]:
results_reranked = get_performance_mrr(df_query_dev, 'cord_uid', 'hybrid_topk')
print(f"Results on the hybrid set: {results_reranked}")

Results on the reranked set: {1: np.float64(0.57), 5: np.float64(0.6207261904761905), 10: np.float64(0.6278222789115646)}


## Use different cross encoder settings on first hybrid retrieve

In [ ]:
from sentence_transformers import CrossEncoder
from tqdm import tqdm

# Initialize Cross-Encoder once
cross_encoder = CrossEncoder('cross-encoder/ms-marco-TinyBERT-L2-v2', max_length=512)

def rank_chunks_with_cross_encoder(query, retrieved_chunks, top_k=5):
    """
    Rank the retrieved chunks based on their relevance to the query using a Cross-Encoder.
    """
    query_chunk_pairs = [[query, chunk] for chunk in retrieved_chunks]
    scores = cross_encoder.predict(query_chunk_pairs)
    ranked_chunks = sorted(zip(retrieved_chunks, scores), key=lambda x: x[1], reverse=True)
    return ranked_chunks[:top_k]

# Initialize new column
df_query_dev["hybrid_cross_encoder_topk"] = None

# Loop over the rows to fill in reranked results
for idx, row in tqdm(df_query_dev.iterrows(), total=len(df_query_dev)):
    query_text = row["tweet_text"]
    topk_ids = row["hybrid_topk"]

    chunk_to_uid = {}
    for uid in topk_ids:
        paper_row = df_collection.loc[df_collection["cord_uid"] == uid]
        if paper_row.empty:
            continue

        title = paper_row["title"].values[0] or ""
        abstract = paper_row["abstract"].values[0] or ""
        full_text = f"{title.strip()}\n{abstract.strip()}"
        chunk_to_uid[full_text] = uid

    reranked_chunks = rank_chunks_with_cross_encoder(query_text, list(chunk_to_uid.keys()))
    reranked_uids = [chunk_to_uid[chunk] for chunk, _ in reranked_chunks]

    # Assign directly to the dataframe
    df_query_dev.at[idx, "hybrid_cross_encoder_topk"] = reranked_uids

# Preview result
print(df_query_dev[["cord_uid", "hybrid_cross_encoder_topk"]].head())

100%|██████████| 1400/1400 [04:05<00:00,  5.71it/s]

   cord_uid                          hybrid_cross_encoder_topk
0  3qvh482o  [nksd3wuw, o4vvlmr4, jrqlhjsm, 8t2tic9n, hg3xp...
1  r58aohnu  [r58aohnu, wk61uyrt, icgsbelo, tu1vevx9, kiq6x...
2  sts48u9i  [u5nxm9tu, 4aps0kvp, osct0lj6, o47v5vgw, sts48...
3  3sr2exq9  [3sr2exq9, k0f4cwig, sv48gjkk, kc3nb2pc, wbw7g...
4  ybwwmyqy  [ybwwmyqy, vabb2f26, 3l6ipiwk, ouvq2wpq, sxx3y...


In [66]:
results_reranked = get_performance_mrr(df_query_dev, 'cord_uid', 'hybrid_cross_encoder_topk')
print(f"Results on the hybrid reranked set: {results_reranked}")

Results on the hybrid reranked set: {1: np.float64(0.5657142857142857), 5: np.float64(0.6185119047619048), 10: np.float64(0.6251488095238095)}


In [68]:
from sentence_transformers import CrossEncoder
from tqdm import tqdm

# Initialize Cross-Encoder once
cross_encoder = CrossEncoder('cross-encoder/ms-marco-TinyBERT-L2-v2', max_length=512)

def rank_chunks_with_cross_encoder(query, retrieved_chunks, top_k=5):
    """
    Rank the retrieved chunks based on their relevance to the query using a Cross-Encoder.
    """
    query_chunk_pairs = [[query, chunk] for chunk in retrieved_chunks]
    scores = cross_encoder.predict(query_chunk_pairs)
    ranked_chunks = sorted(zip(retrieved_chunks, scores), key=lambda x: x[1], reverse=True)
    return ranked_chunks[:top_k]

# Initialize new column
df_query_dev["hybrid_cross_encoder_top5"] = None

# Loop over the rows to fill in reranked results
for idx, row in tqdm(df_query_dev.iterrows(), total=len(df_query_dev)):
    query_text = row["tweet_text"]
    topk_ids = row["hybrid_topk"]

    chunk_to_uid = {}
    for uid in topk_ids:
        paper_row = df_collection.loc[df_collection["cord_uid"] == uid]
        if paper_row.empty:
            continue

        title = paper_row["title"].values[0] or ""
        abstract = paper_row["abstract"].values[0] or ""
        full_text = f"{title.strip()}\n{abstract.strip()}"
        chunk_to_uid[full_text] = uid

    reranked_chunks = rank_chunks_with_cross_encoder(query_text, list(chunk_to_uid.keys()))
    reranked_uids = [chunk_to_uid[chunk] for chunk, _ in reranked_chunks]

    # Assign directly to the dataframe
    df_query_dev.at[idx, "hybrid_cross_encoder_top5"] = reranked_uids

# Preview result
print(df_query_dev[["cord_uid", "hybrid_cross_encoder_top5"]].head())

100%|██████████| 1400/1400 [04:06<00:00,  5.68it/s]

   cord_uid                          hybrid_cross_encoder_top5
0  3qvh482o  [nksd3wuw, o4vvlmr4, jrqlhjsm, 8t2tic9n, hg3xp...
1  r58aohnu  [r58aohnu, wk61uyrt, icgsbelo, tu1vevx9, kiq6x...
2  sts48u9i  [u5nxm9tu, 4aps0kvp, osct0lj6, o47v5vgw, sts48...
3  3sr2exq9  [3sr2exq9, k0f4cwig, sv48gjkk, kc3nb2pc, wbw7g...
4  ybwwmyqy  [ybwwmyqy, vabb2f26, 3l6ipiwk, ouvq2wpq, sxx3y...


In [69]:
results_reranked = get_performance_mrr(df_query_dev, 'cord_uid', 'hybrid_cross_encoder_top5')
print(f"Results on the hybrid reranked set: {results_reranked}")

Results on the hybrid reranked set: {1: np.float64(0.5657142857142857), 5: np.float64(0.6185119047619048), 10: np.float64(0.6185119047619048)}


In [70]:
from sentence_transformers import CrossEncoder
from tqdm import tqdm

# Initialize Cross-Encoder once
cross_encoder = CrossEncoder('cross-encoder/ms-marco-MiniLM-L2-v2', max_length=512)

def rank_chunks_with_cross_encoder(query, retrieved_chunks, top_k=10):
    """
    Rank the retrieved chunks based on their relevance to the query using a Cross-Encoder.
    """
    query_chunk_pairs = [[query, chunk] for chunk in retrieved_chunks]
    scores = cross_encoder.predict(query_chunk_pairs)
    ranked_chunks = sorted(zip(retrieved_chunks, scores), key=lambda x: x[1], reverse=True)
    return ranked_chunks[:top_k]

# Initialize new column
df_query_dev["hybrid_minilm_cross_encoder_topk"] = None

# Loop over the rows to fill in reranked results
for idx, row in tqdm(df_query_dev.iterrows(), total=len(df_query_dev)):
    query_text = row["tweet_text"]
    topk_ids = row["hybrid_topk"]

    chunk_to_uid = {}
    for uid in topk_ids:
        paper_row = df_collection.loc[df_collection["cord_uid"] == uid]
        if paper_row.empty:
            continue

        title = paper_row["title"].values[0] or ""
        abstract = paper_row["abstract"].values[0] or ""
        full_text = f"{title.strip()}\n{abstract.strip()}"
        chunk_to_uid[full_text] = uid

    reranked_chunks = rank_chunks_with_cross_encoder(query_text, list(chunk_to_uid.keys()))
    reranked_uids = [chunk_to_uid[chunk] for chunk, _ in reranked_chunks]

    # Assign directly to the dataframe
    df_query_dev.at[idx, "hybrid_minilm_cross_encoder_topk"] = reranked_uids

# Preview result
print(df_query_dev[["cord_uid", "hybrid_minilm_cross_encoder_topk"]].head())

100%|██████████| 1400/1400 [16:26<00:00,  1.42it/s]

   cord_uid                   hybrid_minilm_cross_encoder_topk
0  3qvh482o  [59up4v56, o4vvlmr4, mb18fj8a, hg3xpej0, jwei2...
1  r58aohnu  [r58aohnu, icgsbelo, h3tor88n, wk61uyrt, eay6q...
2  sts48u9i  [osct0lj6, tz2shoso, 3xw4qjoy, azl5dj74, gruir...
3  3sr2exq9  [3sr2exq9, k0f4cwig, sv48gjkk, 9mdf927z, 35xql...
4  ybwwmyqy  [ouvq2wpq, rs3umc1x, lzddnb8j, gzhc0x44, vabb2...


In [71]:
results_reranked = get_performance_mrr(df_query_dev, 'cord_uid', 'hybrid_minilm_cross_encoder_topk')
print(f"Results on the hybrid reranked set: {results_reranked}")

Results on the hybrid reranked set: {1: np.float64(0.4785714285714286), 5: np.float64(0.5498333333333333), 10: np.float64(0.5584163832199547)}


## Use different retrieval approaches

In [72]:
import numpy as np
from sentence_transformers import util


def encoder_retrieve(query, corpus_embeddings, cord_uids, top_k=20):
    # Step 1: Get dense (bi-encoder) similarity scores
    query_embedding = bi_encoder.encode(query, convert_to_tensor=True)
    dense_scores = util.cos_sim(query_embedding, corpus_embeddings)[0].cpu().numpy()  # shape: (N,)

    # Step 2: Get top-k documents based on hybrid score
    top_indices = np.argsort(-dense_scores)[:top_k]
    top_docs = [cord_uids[i] for i in top_indices]

    return top_docs


In [73]:
df_query_dev["dense_topk"] = None

# Apply hybrid retrieval to each row
for idx, row in tqdm(df_query_dev.iterrows(), total=len(df_query_dev)):
    query_text = row["tweet_text"]
    top_docs = encoder_retrieve(query_text, corpus_embeddings, cord_uids, top_k=20)
    df_query_dev.at[idx, "dense_topk"] = top_docs

100%|██████████| 1400/1400 [00:28<00:00, 49.91it/s]


In [74]:
results_reranked = get_performance_mrr(df_query_dev, 'cord_uid', 'dense_topk')
print(f"Results on the dense set: {results_reranked}")

Results on the dense set: {1: np.float64(0.4157142857142857), 5: np.float64(0.48966666666666664), 10: np.float64(0.4993826530612245)}


For now, we can observe that the hybrid retrieval approach outperforms all other approaches